# Extracting table header as list

In [133]:
import collections
collections.Callable = collections.abc.Callable

In [134]:
from bs4 import BeautifulSoup, NavigableString
from html_table_extractor.extractor import Extractor
import pandas as pd
import os
import re

In [135]:
path = "/home/riya/Downloads/ADT_2019-20/Output_html_Chapter2/kaggle/working/out/output/"

In [136]:
os.chdir(path)

In [137]:
os.listdir()

['output_page_18.html',
 'output_page_15.html',
 'output_page_10.html',
 'output_page_16.html',
 'output_page_1.html',
 'output_page_29.html',
 'output_page_2.html',
 'output_page_5.html',
 'output_page_11.html',
 'output_page_8.html',
 'output_page_31.html',
 'output_page_21.html',
 'output_page_27.html',
 'output_page_19.html',
 'output_page_3.html',
 'output_page_13.html',
 'output_page_20.html',
 'output_page_4.html',
 'output_page_22.html',
 'output_page_23.html',
 'output_page_25.html',
 'output_page_14.html',
 'output_page_28.html',
 'output_page_9.html',
 'output_page_26.html',
 'output_page_12.html',
 'output_page_17.html',
 'output_page_7.html',
 'output_page_30.html',
 'output_page_6.html',
 'output_page_24.html']

In [175]:
output_list = []

In [176]:
output_list

[]

In [177]:
allowed_two_letter_words = {'sl','sc', 'st', 'cc', 'tp', 'tb', 'ts', 'tg', 'km', 'gm', 'an', 'at', 'by', 'do', 'go', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us', 'mm'}

def replace_td_content(tag):
    if isinstance(tag, NavigableString):
        return  # Skip strings, as they don't have 'contents'

    if tag.name == 'td':
        # Extract only meaningful English words and numbers
        valid_characters = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.%")
        filtered_content = ''.join(char if char in valid_characters else ' ' for char in tag.get_text())
        
        # Remove extra spaces and words with less than 3 characters
        cleaned_content = ' '.join(word for word in filtered_content.split() if len(word) >= 3 or word.isdigit() or word.lower() in allowed_two_letter_words or word=='%')
        
        # Set the cleaned content as the tag's string
        tag.string = cleaned_content
    elif tag.contents:
        for child in tag.contents:
            replace_td_content(child)


In [178]:
def process_table_cells(target_table):
    if not target_table:
        print("Table not found in the HTML content.")
        return

    flag1 = 0
    flag2 = 0

    # Loop through each row in the table
    for row in target_table.find_all('tr'):
        # Loop through each cell in the row
        if(flag1 == 1 and flag2 == 1):
            row.decompose()
        else : 
            for cell in row.find_all(['td', 'th']):
                # Process the cell content (you can modify this part based on your requirements)
                cell_content = cell.get_text(strip=True)
                if(cell_content == '1'):
                    flag1 = 1
                if(cell_content == '2'):
                    flag2 = 1        
            

In [179]:
# iterate through all file 
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith(".html"): 
        file_path = f"{path}{file}"
        with open(file_path) as fp:
            soup = BeautifulSoup(fp, "html.parser")
            
            for tag in soup.find_all(attrs={'style': True}):
                tag.attrs.pop('style')
            
            table_tag = soup.find('table')
            
            if table_tag is not None:
                replace_td_content(table_tag)

                process_table_cells(table_tag)

                extractor1 = Extractor(table_tag)
                extractor1.parse()
                output_list.append(extractor1.return_list())
            else:
                print(file_path)

/home/riya/Downloads/ADT_2019-20/Output_html_Chapter2/kaggle/working/out/output/output_page_17.html
/home/riya/Downloads/ADT_2019-20/Output_html_Chapter2/kaggle/working/out/output/output_page_24.html


In [180]:
output_list

[[['2.12', '2.12', '2.12', '2.12', '2.12', '2.12', '2.12'],
  ['Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers'],
  ['Sl No', 'Districts', '2017', '2018', '2019', '2020', '2021'],
  ['1', '2', '189', '190', '191', '192', '193']],
 [['2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021'],
  ['Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise 

In [181]:
def contain_table_id(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'(?<!\S)(?:' + '(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})' + r')(?!\S)', flags=re.IGNORECASE)

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    if len(matches) == 0:
        return False
    return True

In [182]:
def are_all_same_id(lst):
    if not lst:
        return False  # An empty list cannot have all same elements.
    first_element = lst[0]
    if first_element != '' and contain_table_id(first_element):
        return all(element == first_element for element in lst[:-1])
    return False

def are_all_same_name(lst):
    if not lst:
        return False  # An empty list cannot have all same elements.
    first_element = lst[0]
    if first_element != '':
        return all(element == first_element for element in lst[:-1])
    return False

In [183]:
def check_for_table(item):
    i=0
    new_item = item
    for k in range(len(item)):
        id_list=[]
        if are_all_same_id(item[k]):
            if i<1:
                i = i+1
            elif i==2:
                return item
        elif i==1 and are_all_same_name(item[k]):
            return item
        elif i==1 and not are_all_same_name(item[k]):
            n_lst = item[k-1]
            new_item.insert(0,n_lst)
            i = i+1
            return new_item
        else:
            new_item.remove(item[k])
            return check_for_table(new_item)

In [184]:
corrected_list=[]
error_list=[]
for inner_list in output_list:
    initial_val = inner_list.copy()
    item = check_for_table(inner_list)
    if item is not None and len(item)>0:
        corrected_list.append(item)
    else:
        error_list.append(initial_val)

In [185]:
corrected_list

[[['2.12', '2.12', '2.12', '2.12', '2.12', '2.12', '2.12'],
  ['Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers'],
  ['Sl No', 'Districts', '2017', '2018', '2019', '2020', '2021'],
  ['1', '2', '189', '190', '191', '192', '193']],
 [['2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021',
   '2.11 2011 2021'],
  ['Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise 

In [186]:
error_list

[[['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of main workers to total workers'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of main workers to total workers'],
  ['1',
   '2',
   '228',
   '229',
   '230',
   '231',
   '232',
   '233',
   '234',
   '235',
   '236',
   '237']],
 [['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of Non workers to total Populati -on'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of Non workers to total Populati -on'],
  ['1',
   '2',
   '248',
   '249',
   '250',
   '251',
   '252',
   '253',
   '254',
   '255',
   '256',
   '257']],
 [['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
  

In [187]:
list_to_remove = [[['2.10 -2011',
   '2.10 -2011',
   '2.10 -2011',
   '2.10 -2011',
   '2.10 -2011',
   '2.10 -2011',
   '2.10 -2011'],
  ['Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census',
   'Age Groupwise Population Rural and Urban per 2011 Census'],
  ['Age Group',
   '',
   '',
   '',
   'to total Population',
   '2021 % Projectedpopulation-2021',
   '2021 Population to 2021 Projected population'],
  ['Age Group',
   'Total',
   'Rural',
   'Urban',
   'to total Population',
   '2021 % Projectedpopulation-2021',
   '2021 Population to 2021 Projected population'],
  ['156', '157', '158', '159', '160', '161', '162'],
  ['0-4', '5046719', '3179495', '1867224', '8.26', '5888130', '8.18'],
  ['5-9', '5241509', '3309619', '1931890', '8.58', '6101489', '8.48'],
  ['10-14', '5736646', '3662475', '2074171', '9.39', '6649360', '9.24'],
  ['15-19', '5827277', '3658264', '2169013', '9.54', '6853227', '9.52'],
  ['20-24', '6052558', '3549425', '2503133', '9.91', '7143613', '9.93'],
  ['25-29', '5771606', '3283508', '2488098', '9.45', '6932776', '9.63'],
  ['30-34', '4698380', '2657749', '2040631', '7.69', '5686264', '7.90'],
  ['35-39', '4695451', '2770958', '1924493', '7.69', '5482112', '7.62'],
  ['40-44', '3826712', '2320355', '1506357', '6.26', '4496270', '6.25'],
  ['45-49', '3519782', '2165676', '1354106', '5.76', '4136924', '5.75'],
  ['50-54', '2699270', '1665456', '1033814', '4.42', '3177704', '4.42'],
  ['55-59', '2142504', '1323457', '819047', '3.51', '2620725', '3.64'],
  ['60-64', '2060052', '1352255', '707797', '3.37', '2405841', '3.34'],
  ['65-69', '1520119', '1039482', '480637', '2.49', '1761458', '2.45'],
  ['70-74', '1061125', '736309', '324816', '1.74', '1221938', '1.70'],
  ['75-79', '524426', '350060', '174366', '0.86', '608287', '0.85'],
  ['80-84', '362024', '249880', '112144', '0.59', '515306', '0.72'],
  ['85-89', '133738', '83752', '49986', '0.22', '103413', '0.14'],
  ['90-94', '74534', '49698', '24836', '0.12', '56620', '0.08'],
  ['95-99', '32257', '21558', '10699', '0.05', '33478', '0.05'],
  ['100', '22757', '14075', '8682', '0.04', '20155', '0.03'],
  ['AgeNotStated', '45851', '25829', '20022', '0.08', '62306', '0.09'],
  ['AlAges',
   'l61095297',
   '37469335',
   '23625962',
   '100.00',
   '71957396',
   '100.00']]] 



In [188]:
for item in list_to_remove:
    corrected_list.remove(item)

In [189]:
list_to_add = [[['2.12', '2.12', '2.12', '2.12', '2.12', '2.12', '2.12', '2.12'],
  ['Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers', 
   'Projected Population In Numbers'],
  ['', '', '2011', '2012', '2013', '2014', '2015', '2016'],
  ['1', '2', '183', '184', '185', '186', '187', '188']],
  [['2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15',
   '2.15'],
  ['Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers',
   'Marginal Workers 2011 Census in Numbers'],
  ['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of marginal workers to total workers'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of marginal workers to total workers'],
  ['1',
   '2',
   '238',
   '239',
   '240',
   '241',
   '242',
   '243',
   '244',
   '245',
   '246',
   '247']],
   [['2.4', '2.4', '2.4', '2.4', '2.4', '2.4', '2.4', '2.4'],
   ['Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population', 
    'Percentage of Child Population to the Total population'],
  ['SI. No',
   'Districts',
   '2011 Census',
   '2011 Census',
   '2011 Census',
   '2021 Projected',
   '2021 Projected',
   '2021 Projected'],
  ['SI. No', 'Districts', '', '', '', '', '', ''],
  ['SI. No',
   'Districts',
   'Total',
   'Rural',
   'Urban',
   'Total',
   'Rural',
   'Urban'],
  ['1', '2', '111', '112', '113', '114', '115', '116']], 
  [['2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14',
   '2.14'],
  ['Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers',
   'Main Workers 2011 Census in Numbers'],
  ['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of main workers to total workers'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of main workers to total workers'],
  ['1',
   '2',
   '228',
   '229',
   '230',
   '231',
   '232',
   '233',
   '234',
   '235',
   '236',
   '237']],
 [['2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16',
   '2.16'],
  ['Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers',
   'Non Workers 2011 Census in Numbers'],
  ['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of Non workers to total Population'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of Non workers to total Population'],
  ['1',
   '2',
   '248',
   '249',
   '250',
   '251',
   '252',
   '253',
   '254',
   '255',
   '256',
   '257']],
   [['2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17',
   '2.17'],
  ['Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers',
   'Agricultural cultivators 2011 Census in Numbers'],
  ['',
   '',
   'Rural',
   'Rural',
   'Rural',
   'Urban',
   'Urban',
   'Urban',
   'Total',
   'Total',
   'Total',
   '% of agricultural cultivators to total workers'],
  ['',
   '',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   'Male',
   'Female',
   'Total',
   '% of agricultural cultivators to total workers'],
  ['1',
   '2',
   '258',
   '259',
   '260',
   '261',
   '262',
   '263',
   '264',
   '265',
   '266',
   '267']]]

In [190]:
corrected_list.extend(list_to_add)

In [191]:
len(output_list)

29

In [192]:
len(corrected_list)

30

In [193]:
len(error_list)

4

In [194]:
def add_space_before_year(input_string):
    # Regular expression to find a year in 'yyyy' format with an optional alphabet character before it
    pattern = re.compile(r'([a-zA-Z])?(\d{4})')
    
    # Function to add space between alphabet character and year
    def replace(match):
        alphabet_part, year_part = match.groups()
        if alphabet_part:
            return f'{alphabet_part} {year_part}'
        else:
            return year_part

    # Using re.sub() to apply the replacement function to the input string
    result = pattern.sub(replace, input_string)

    return result

In [195]:
def remove_before_all_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    for i in range(len(matches)):
        second_occurrence_index = matches[i].end()
        input_string = input_string[second_occurrence_index:]

    return input_string.strip()

In [196]:
def remove_after_all_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    for i in range(len(matches)):
        second_occurrence_index = matches[i].end()
        input_string = input_string[:second_occurrence_index]

    return input_string.strip()

In [197]:
def convert_uppercase_to_camel_case(input_string):
    words = input_string.split()
    result_words = [word.capitalize() if word.isupper() else word for word in words]
    result_string = ' '.join(result_words)
    return result_string

In [198]:
def add_space_before_camelcase(input_string):
    # Use regular expression to find camel case words
    camelcase_pattern = re.compile(r'(?<=[a-z])(?=[A-Z])')
    result_string = camelcase_pattern.sub(' ', input_string)
    return result_string

In [199]:
def remove_extra_spaces(input_string):
    # Use regular expression to replace consecutive white spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    return cleaned_string.strip()

In [200]:
def add_space_before_word_with_condition(input_string):
    # Use regular expression to find words preceded by a number or special character
    pattern = re.compile(r'(?<=[%])\b(\w+)\b', flags=re.IGNORECASE)

    # Using re.sub() to add space before matched words
    result_string = pattern.sub(lambda x: ' ' + x.group(1), input_string)

    return result_string

In [201]:
def replace_words_with_blank(input_string, words_to_replace):
    # Constructing a regular expression pattern to match any of the words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')\b', flags=re.IGNORECASE)
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')(?!\S)', flags=re.IGNORECASE)

    # Using re.sub() to replace matched words with blank
    result = pattern.sub('', input_string)

    return result.strip()

In [202]:
words_to_replace = ['201 1', '0-6 20', '11 0-6', '-2011', '0-6 2011', '0-6 2001', '0-6 2001 2011', '2001 and 2011', '20 01', '2011 2021', '31.03', '.CA', '.Dg', '.G.S', '.Jfl', '.Q.', '.ffl', '1 2', '1-10', '2 01', '2006 jAz 2016', '2011 twAi rAi AaP fiU', '2011 twAi zsj wAi EAzs %', '20112.2', '20112.6', '2011g', '24 7', '31.03.2020', '31.3.202', '31.3.2020', '31.3.2020.', '33 jAz 6', '6 jAz 1', '6 jAz 14', '6 wAU', 'A.ffl', 'A.g', 'AdP', 'Adg', 'Affl', 'Afi', 'Afli', 'Aid', 'Arg', 'Azs', 'CAU', 'CAZ', 'CAv', 'CAw', 'CAz', 'Cfi', 'CrP', 'Dfi', 'Dfl', 'E.J .n. .Dg .Jfl .JA.', 'EAf', 'Efi', 'Efl', 'JfiP', 'Jfl', 'KfiQ', 'MlM', 'Pfi', 'QAi', 'Raing', 'Rfl', 'SQA', 'Three', 'VAi', 'dfl', 'eng cy.U', 'ffl', 'fiAi', 'fiP', 'fiQ', 'fiQAi', 'fiU', 'fig', 'fl.', 'gLi', 'gMl', 'jAU', 'jAi', 'lion nit', 'ofD', 'rAi', 'tAwAi', 'twAi', 'tzx', 'wAi', 'tzs']

In [203]:
def replace_words(input_string, word_replacements):
    # Constructing a regular expression pattern to match whole words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')\b')
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')(?!\S)')

    # Using re.sub() to replace matched words with their corresponding replacements
    result = pattern.sub(lambda x: word_replacements.get(x.group(), x.group()), input_string)

    return result

In [204]:
word_replacements = {"2011Sex" : "Sex","2001 Sex" : "Sex", "15 59" : "15-59", "2001-2011 Decadal" : "Decadal", "2011 Scheduled" : "Scheduled", "0 6" : "0-6", "2001-2011Decadal" : "Decadal", "2011 Population" : "Population", "2001 11Decadal"  : "Decadal", "0 14" : "0-14", "2001 2011 Decadal" : "Decadal", "Agricult -ural cultivato -rs" : "Agricultural cultivators", "2001 %" : "%", "2011 %" : "%", "ank" : "Rank", "2020 Projected" : "Projected", "Cen" : "Census", "2001 11" : "2001-11", "2Child" : "Child", "-Projected" : "Projected", "6" : "0-6", "Agricult -ural labour -ers" : "Agricultural labourers", "ofworkers" : "of workers", "house -hold" : "household", "ofother" : "of other", "totalp ulati -on" : "total poulation", "agricu -ltural" : "agricultural", "2011 Religion-wise" : "Religion-wise", "2011 Workers" : "Workers", "2011 Tota" : "Total", "2011 Non-" : "Non-", "2011 Area" : "Area"}

In [205]:
def process_data(input_data):
    for k in range(len(input_data)):
        for i in range(len(input_data[k])):
            for j in range(len(input_data[k][i])):
                #print(k,i,j,input_data[k][i][j])

                input_data[k][i][j] = add_space_before_year(input_data[k][i][j])
                input_data[k][i][j] = replace_words_with_blank(input_data[k][i][j], words_to_replace)
                input_data[k][i][j] = replace_words(input_data[k][i][j], word_replacements)
                input_data[k][i][j] = add_space_before_camelcase(input_data[k][i][j])
                input_data[k][i][j] = convert_uppercase_to_camel_case(input_data[k][i][j])
                input_data[k][i][j] = remove_extra_spaces(input_data[k][i][j])
                input_data[k][i][j] = add_space_before_word_with_condition(input_data[k][i][j])

In [206]:
process_data(corrected_list)
corrected_list

[[['2.12', '2.12', '2.12', '2.12', '2.12', '2.12', '2.12'],
  ['Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers',
   'Projected Population In Numbers'],
  ['Sl No', 'Districts', '2017', '2018', '2019', '2020', '2021'],
  ['1', '2', '189', '190', '191', '192', '193']],
 [['2.11', '2.11', '2.11', '2.11', '2.11', '2.11', '2.11', '2.11'],
  ['Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Age wise Population per 2011 Census

In [207]:
#transpose the inner list
transp_list = [[list(pair) for pair in zip(*sublist)] for sublist in corrected_list]
transp_list

[[['2.12', 'Projected Population In Numbers', 'Sl No', '1'],
  ['2.12', 'Projected Population In Numbers', 'Districts', '2'],
  ['2.12', 'Projected Population In Numbers', '2017', '189'],
  ['2.12', 'Projected Population In Numbers', '2018', '190'],
  ['2.12', 'Projected Population In Numbers', '2019', '191'],
  ['2.12', 'Projected Population In Numbers', '2020', '192'],
  ['2.12', 'Projected Population In Numbers', '2021', '193']],
 [['2.11',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Si. No',
   'Si. No',
   'Si. No',
   '1'],
  ['2.11',
   'Age wise Population per 2011 Census and Projected population of 2021',
   'Districts',
   'Districts',
   'Districts',
   '2'],
  ['2.11',
   'Age wise Population per 2011 Census and Projected population of 2021',
   '15-59 2011',
   '',
   'Male',
   '169'],
  ['2.11',
   'Age wise Population per 2011 Census and Projected population of 2021',
   '15-59 2011',
   '',
   'Female',
   '170'],
  ['2.11',
   'Age w

In [208]:
def find_table_details(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = pattern.finditer(input_string)

    # Initialize variables to store the matches
    match_positions = []

    # Store the positions of the matches
    for match in matches:
        match_positions.append(match.start())

    # If there is at least one occurrence of the pattern
    if match_positions:
        table_id_index = match_positions[0]
        input_string = input_string[table_id_index:]  
        space_index = input_string.find(' ') 
        table_id = input_string[table_id_index:space_index].strip()
        table_name = input_string[space_index:].strip()
        return table_id, table_name    
    else:
        return None, None


In [209]:
def convert_to_number(string):
    try:
        number = int(string)
    except ValueError:
        try:
            number = float(string)
        except ValueError:
            print(string)
            raise ValueError("Input string is not a valid number")
    return number

In [210]:
chapter_id = 2
chapter_name = "Area and Population"
#processing each column
def process_dictionary(input_data):
    df = pd.DataFrame(columns = ['Attr_id', 'Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description'])
    for inner_list in input_data:
        table_id = remove_after_all_occurrence(inner_list[0][0])
        table_name = remove_before_all_occurrence(inner_list[0][1])
        previd=0
        for data_list in inner_list:
            i=0
            attr_id=0
            description = ''
            prev = ''
            for data in reversed(data_list):
                if data == '1' or data == '2':
                    break
                i = i+1;
                if i == 1 and data!='' and (previd==0 or convert_to_number(data)!=previd):
                    attr_id = convert_to_number(data)
                    previd = convert_to_number(data)
                if i > 1 and i < len(data_list)-1:
                    if description == '':
                        description = data
                        prev = data
                    else : 
                        if prev != data and data!='':
                            description = ','.join([description, data]) #description + ' ' + data
                            prev = data
            if attr_id != 0 and description != "" :
                row = {'Attr_id' : attr_id, 'Chapter_id' : chapter_id, 'Chapter_name' : chapter_name, 'Table_id' : table_id, 'Table_name' : table_name, 'Description' : remove_extra_spaces(description)}
                df.loc[len(df)] = row
    df = df.sort_values(by=['Attr_id'])
    df.to_csv('/home/riya/Downloads/ADT_2019-20/Output_html_Chapter2/Chapter2_Attributes.csv', sep=';', index=False)
    df.to_csv('/home/riya/Downloads/ADT_2019-20/Dictionary/Chapter2_Attributes.csv', sep=';', index=False)
                                            

In [211]:
process_dictionary(transp_list)